In [1]:
import os

In [2]:
%pwd

'c:\\Users\\farde\\YouTube\\End to end ml project\\End-to-end-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\farde\\YouTube\\End to end ml project\\End-to-end-ML-Project'

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [40]:
from mlproject.constants import *
from mlproject.utlis.common import read_yaml,create_directories

In [42]:
class ConfigurationManger:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories=([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

In [43]:
import pandas as pd
import os
from mlproject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [59]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        train_x=train_data.drop([self.config.target_column],axis=1)
        test_x=test_data.drop([self.config.target_column],axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]


        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)
        print(f"{self.config.root_dir}/"+self.config.model_name)
        print(os.path.join(self.config.root_dir,self.config.model_name))
              
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

In [74]:
ModelTrainer

__main__.ModelTrainer

In [60]:
try:
    config=ConfigurationManger()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-04-04 17:09:00,183:INFO:common:yaml file: config\config.yaml loaded sucessfully]
[2024-04-04 17:09:00,187:INFO:common:yaml file: params.yaml loaded sucessfully]
[2024-04-04 17:09:00,192:INFO:common:yaml file: schema.yaml loaded sucessfully]
[2024-04-04 17:09:00,196:INFO:common:created directory at: artifacts]
artifacts/model_trainer/model.joblib
artifacts/model_trainer\model.joblib


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/model_trainer\\model.joblib'

In [48]:
os.path.join("artifacts",'model.pkl')

'artifacts\\model.pkl'